# Extracting Database Schema from PostgreSQL

In [32]:
import psycopg2
import json



# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="123456",
    host="localhost",
    port="5432"
)


cursor = conn.cursor()

# Get tables
cursor.execute("""
    SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';
""")
tables = [row[0] for row in cursor.fetchall()]

# Get columns
schema = []
for table in tables:
    cursor.execute(f"""
        SELECT column_name, data_type 
        FROM information_schema.columns 
        WHERE table_name = '{table}';
    """)
    columns = [{"name": row[0], "type": row[1]} for row in cursor.fetchall()]
    
    schema.append({"name": table, "columns": columns})

# Get foreign keys
cursor.execute("""
    SELECT 
        conrelid::regclass AS table_name, 
        a.attname AS column_name,
        confrelid::regclass AS foreign_table_name
    FROM pg_constraint AS c
    JOIN pg_attribute AS a 
    ON a.attnum = ANY(c.conkey) AND a.attrelid = c.conrelid
    WHERE c.contype = 'f';
""")
foreign_keys = cursor.fetchall()
for fk in foreign_keys:
    for table in schema:
        if table["name"] == fk[0]:
            for column in table["columns"]:
                if column["name"] == fk[1]:
                    column["foreign_key"] = f"{fk[2]}.{fk[1]}"

# Close connection
cursor.close()
conn.close()

# Output schema as JSON
schema_json = json.dumps({"tables": schema}, indent=4)
print(schema_json)


{
    "tables": [
        {
            "name": "packages_apt_postgresql_org",
            "columns": [
                {
                    "name": "id",
                    "type": "integer"
                },
                {
                    "name": "last_updated",
                    "type": "timestamp without time zone"
                },
                {
                    "name": "aptdata",
                    "type": "jsonb"
                }
            ]
        },
        {
            "name": "packages_yum_postgresql_org",
            "columns": [
                {
                    "name": "id",
                    "type": "integer"
                },
                {
                    "name": "last_updated",
                    "type": "timestamp without time zone"
                },
                {
                    "name": "yumdata",
                    "type": "jsonb"
                }
            ]
        },
        {
            "name": "film_actor"

In [33]:
import json 
with open('schema.json', 'w') as f:
    f.write(schema_json)    

# Creating Summary Metadata

In [ ]:
import os
from google import genai
from google.genai import types
from dotenv import load_dotenv
from app.database import *
load_dotenv()

True

In [4]:
client = genai.Client(api_key=os.getenv('API_KEY'))

In [64]:
# !pip install pybase64

In [23]:
import base64
image_path = r'C:\Users\Anush\Desktop\Projects\text-to-sql\pagila\pagila-schema-diagram.png'
with open(image_path, 'rb') as file:
    img_b = file.read()
img_b64 = base64.b64encode(img_b).decode('utf-8')

In [ ]:
template = """
    Database: Pagila.
    Use the image, and the database schema extracted from PostreSQL to generate structured descriptive metadata that will be fed to models for building a Text-to-SQL application using LLMs.
    Think step by step and carefully analyse both the things to create descriptions for tables, their columns and especially the relationships between tables, so that the models can use it as a reference when the user asks questions in Natural Language, the model can generate accurate SQL quries.
    === Schema
    {schema}
    === Continue From Where you left off and when finished literally just write "{stop}" and I'll use it to stop the loop so you can go into good detail and leave it there and resume from where you left off!
    {checkpoint}
    """


In [63]:
img_obj =  types.Part.from_bytes(data=img_b64, mime_type="image/png")

In [70]:
from time import sleep

stop = "DZl6r5WmcbPKzvH"
values = {
    'schema' : schema_json,
    'checkpoint' : '/begin',
    'stop' : stop
}
metadata = ""

prompt = template.format(**values)
contents = [prompt, img_obj]
response = client.models.generate_content(model='gemini-2.0-flash-001', contents=contents, config={'temperature' : 0})


while stop not in response.text  :
    sleep(0.5)
    metadata += response.text[3:-3].replace('json', '') # removing '''json  '''
    prompt = template.format(**values)
    contents = [prompt, img_obj]
    response = client.models.generate_content(model='gemini-2.0-flash-001', contents=contents, config={'temperature' : 0})
    values = {
        'schema' : schema_json,
        'checkpoint' : metadata,
        'stop': stop
    }

In [69]:
response.text

"Okay, I'm going to create detailed descriptions for each table and column in the Pagila database, focusing on relationships and data types to aid in Text-to-SQL generation.\n\n**packages_apt_postgresql_org**\n*   Description: This table stores information about PostgreSQL packages available through APT (Debian/Ubuntu) repositories.\n*   Columns:\n    *   `id` (integer): Primary key for the table.\n    *   `last_updated` (timestamp without time zone): Timestamp indicating when the package information was last updated.\n    *   `aptdata` (jsonb): JSONB column storing the package information retrieved from the APT repository.\n\n**packages_yum_postgresql_org**\n*   Description: This table stores information about PostgreSQL packages available through YUM (Red Hat/CentOS) repositories.\n*   Columns:\n    *   `id` (integer): Primary key for the table.\n    *   `last_updated` (timestamp without time zone): Timestamp indicating when the package information was last updated.\n    *   `yumdata